In [2]:
import pandas as pd
import csv
import re
import time
import json
import random
import requests
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ==========================================
# 0. 설정값
# ==========================================
CSV_PATH = "musinsa_belt_ids.csv"   # product_id 컬럼 포함 CSV

CATEGORY_ID = 101007        # img_url용 PLP 카테고리
PLP_MAX_PAGES = 200

DB_HOST = "musinsa-db.c07kuo6ug98z.us-east-1.rds.amazonaws.com"
DB_NAME = "musinsa"
DB_USER = "admin"
DB_PASSWORD = "qkqajrwkrnrnrn9_"
DB_PORT = 3306

# ==========================================
# 1. product_id 로드
# ==========================================
product_ids = []
with open(CSV_PATH, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row.get("goodsNo"):
            product_ids.append(int(row["goodsNo"]))

product_ids = list(dict.fromkeys(product_ids))[94:]  # 중복 제거
print(f"총 product_id 수: {len(product_ids)}")

# ==========================================
# 2. PLP API → goodsNo : thumbnail 매핑
# ==========================================
def fetch_goods_thumbnail_map(category_id, max_pages=100, size=60):
    api_url = "https://api.musinsa.com/api2/dp/v1/plp/goods"
    goods_map = {}

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://www.musinsa.com/",
    }

    for page in range(1, max_pages + 1):
        params = {
            "gf": "M",
            "sortCode": "SALE_ONE_YEAR_COUNT",
            "category": category_id,
            "size": size,
            "testGroup": "",
            "caller": "CATEGORY",
            "page": page,
            "seen": 0,
            "seenAds": ""
        }

        res = requests.get(api_url, params=params, headers=headers, timeout=10)

        if res.status_code == 403:
            print(f"⛔ PLP API 403 차단 (page={page})")
            break

        goods_list = res.json().get("data", {}).get("list", [])
        if not goods_list:
            break

        for g in goods_list:
            goods_no = g.get("goodsNo")
            thumbnail = g.get("thumbnail")
            if goods_no and thumbnail:
                goods_map[int(goods_no)] = thumbnail

        time.sleep(0.3)

    return goods_map

print("PLP API에서 img_url 수집 중...")
goods_thumbnail_map = fetch_goods_thumbnail_map(CATEGORY_ID)
print(f"img_url 매핑 수: {len(goods_thumbnail_map)}")

# ==========================================
# 3. Selenium 설정
# ==========================================
options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.set_page_load_timeout(20)

# ==========================================
# 4. DB 연결
# ==========================================
conn = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    db=DB_NAME,
    port=DB_PORT,
    charset="utf8mb4"
)
cursor = conn.cursor()

insert_sql = """
INSERT INTO product_new (
    product_id, product_name, brand,
    original_price, sale_price,
    upper_category, lower_category,
    gender, rating,
    wish_count, review_count,
    size_info, discount_rate,
    fit_season, cumulative_sales,
    style, img_url
) VALUES (
    %s, %s, %s, %s, %s,
    %s, %s, %s, %s,
    %s, %s, %s, %s,
    %s, %s, %s, %s
)
"""

# ==========================================
# 5. 공통 파싱 함수
# ==========================================
def extract_int(text):
    return int(re.sub(r"[^0-9]", "", text)) if text else None

def extract_number(text):
    if not text:
        return None
    if "만" in text:
        try:
            return int(float(text.replace("만", "")) * 10000)
        except:
            return None
    cleaned = re.sub(r"[^0-9]", "", text)
    return int(cleaned) if cleaned else None

def clean_product_name(name):
    return re.sub(r"-.*", "", name).strip()

# ==========================================
# 6. 크롤링 → 즉시 DB INSERT
# ==========================================
print("크롤링 + DB INSERT 시작!")

for idx, product_id in enumerate(product_ids, start=1):
    print(f"[{idx}/{len(product_ids)}] product_id={product_id}")

    try:
        driver.get(f"https://www.musinsa.com/products/{product_id}")
        time.sleep(random.uniform(0.8, 1.5))
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 상품명 / 브랜드
        name_tag = soup.select_one("div[class*='GoodsName__Wrap'] span")
        product_name = name_tag.get_text(strip=True)

        brand_tag = soup.select_one("div[class*='Brand__Wrap'] span[class*='BrandName']")
        brand = brand_tag.get_text(strip=True) if brand_tag else None

        if brand:
            brand = brand.replace("단독", "").strip()

        # 가격
        discount_rate = extract_int(
            soup.select_one("span[class*='DiscountRate']").get_text()
        ) if soup.select_one("span[class*='DiscountRate']") else None

        if discount_rate is not None:
            original_price = extract_int(
                soup.select_one("span.line-through").get_text()
            )
            sale_price = extract_int(
                soup.select_one("span[class*='CalculatedPrice']").get_text()
            )
        else:
            original_price = extract_int(
                soup.select_one("span[class*='CalculatedPrice']").get_text()
            )
            sale_price = None

        # 카테고리
        upper_category = None
        lower_category = None
        cat_wrap = soup.select_one("div[class*='Category__Wrap']")
        if cat_wrap:
            for a in cat_wrap.find_all("a"):
                if a.get("data-category-id") == "1depth":
                    upper_category = a.get_text(strip=True)
                elif a.get("data-category-id") in ("2depth", "3depth"):
                    lower_category = a.get_text(strip=True)

        # 성별 / 누적판매
        gender = 0
        cumulative_sales = None
        info_boxes = soup.select("dl[class*='Layout__Wrap'] div[class*='Layout__Box']")
        for box in info_boxes:
            dt = box.find("dt")
            dd = box.find("dd")
            if not dt or not dd:
                continue
            if dt.get_text(strip=True) == "성별":
                gender = {"남":1,"여":2,"공용":0,"남녀공용":0}.get(dd.get_text(strip=True),0)
            if dt.get_text(strip=True) == "누적판매":
                cumulative_sales = dd.get_text(strip=True)

        # 별점 / 후기 / 관심
        rating = None

        for span in soup.find_all("span"):
            class_list = span.get("class", [])

            # 별점이 있는 경우에만 존재하는 정확한 class 조합
            if (
                "text-body_13px_med" in class_list
                and "text-black" in class_list
                and "font-pretendard" in class_list
            ):
                try:
                    rating = float(span.get_text(strip=True))
                    break
                except:
                    rating = None
        
        review_count = extract_number(
            soup.select_one("div[class*='Review__Wrap'] span:nth-of-type(2)").get_text()
        ) if soup.select_one("div[class*='Review__Wrap'] span:nth-of-type(2)") else 0

        wish_count = extract_number(
            soup.select_one("div[class*='Like__Container'] span").get_text()
        ) if soup.select_one("div[class*='Like__Container'] span") else None

        # 스타일
        style = ",".join(
            t.get_text(strip=True)
            for t in soup.select("ul[class*='ProductTags__List'] span")
        )

        # img_url (PLP API)
        img_url = goods_thumbnail_map.get(product_id, "")

        # DB INSERT
        cursor.execute(
            insert_sql,
            (
                product_id, product_name, brand,
                original_price, sale_price,
                upper_category, lower_category,
                gender, rating,
                wish_count, review_count,
                json.dumps([], ensure_ascii=False),
                discount_rate,
                json.dumps({"핏": [], "계절감": []}, ensure_ascii=False),
                cumulative_sales,
                style,
                img_url
            )
        )
        conn.commit()

    except Exception as e:
        print(f"❌ 실패: product_id={product_id}, {e}")

# ==========================================
# 7. 종료
# ==========================================
driver.quit()
cursor.close()
conn.close()

print("✅ 전체 크롤링 + DB INSERT 완료")


총 product_id 수: 406
PLP API에서 img_url 수집 중...
img_url 매핑 수: 1987
크롤링 + DB INSERT 시작!
[1/406] product_id=4726867
[2/406] product_id=4246354
[3/406] product_id=4255731
[4/406] product_id=1224998
[5/406] product_id=2305801
[6/406] product_id=2584858
[7/406] product_id=3489621
[8/406] product_id=1961480
[9/406] product_id=4304066
❌ 실패: product_id=4304066, (1062, "Duplicate entry '4304066' for key 'product_new.PRIMARY'")
[10/406] product_id=1595795
[11/406] product_id=4043128
[12/406] product_id=1031296
[13/406] product_id=4345136
[14/406] product_id=4342330
[15/406] product_id=4051365
[16/406] product_id=4797520
[17/406] product_id=5286599
[18/406] product_id=3767800
[19/406] product_id=4201333
[20/406] product_id=4760773
[21/406] product_id=3839872
[22/406] product_id=4246350
[23/406] product_id=1031295
[24/406] product_id=5238796
[25/406] product_id=5056657
[26/406] product_id=3594393
[27/406] product_id=1224173
[28/406] product_id=4342335
[29/406] product_id=3113079
[30/406] product_id=